In [104]:
import numpy as np

In [105]:
name_dict = { "плохая" : 0.1, "средняя" : 0.4, "хорошая" : 0.73, "развитая" : 0.9 }

In [106]:
table = [
    [name_dict["средняя"], name_dict["хорошая"], 3.5],
    [name_dict["плохая"], name_dict["хорошая"], 2.5],
    [name_dict["развитая"], name_dict["плохая"], 3],
    [name_dict["развитая"]+0.1, name_dict["средняя"], 3.5],
    [name_dict["средняя"], name_dict["хорошая"] + 0.5, 3],
    [name_dict["плохая"], name_dict["средняя"], 2]
]

print(table)

[[0.4, 0.73, 3.5], [0.1, 0.73, 2.5], [0.9, 0.1, 3], [1.0, 0.4, 3.5], [0.4, 1.23, 3], [0.1, 0.4, 2]]


In [107]:
def Pareto_method(table):
    Y = table.copy()
    delete_indices = []
    for i in range(len(table)-1):
        forDelete = True
        for k in range(len(table[i])):
            if table[i][k] < table[i+1][k]:
                forDelete = False
                break
        
        if forDelete:
            Y.pop(i+1-(len(table)-len(Y)))
        else:
            forDelete = True
            for k in range(len(table[i])):
                if table[i][k] >= table[i+1][k]:
                    forDelete = False
                    break
        
            if forDelete:
                Y.pop(i-(len(table)-len(Y)))

    return Y


In [108]:
pareto_table = Pareto_method(table)
print("Before Pareto: {}".format(table))
print("---------------------------------------------------------------------------------------")
print("After Pareto: {}".format(pareto_table))

Before Pareto: [[0.4, 0.73, 3.5], [0.1, 0.73, 2.5], [0.9, 0.1, 3], [1.0, 0.4, 3.5], [0.4, 1.23, 3], [0.1, 0.4, 2]]
---------------------------------------------------------------------------------------
After Pareto: [[0.4, 0.73, 3.5], [1.0, 0.4, 3.5], [0.4, 1.23, 3]]


In [109]:
rating = [[9, 7, 7],[9, 8, 7]]

In [110]:
def rank_method(rating):
    rating = np.array(rating)
    c_sum = [sum(rating[:, i]) for i in range(len(rating[0]))]
    C = sum(c_sum)
    weights = [c_sum[i] / C for i in range(len(c_sum))]
    return weights

In [111]:
weights = rank_method(rating)
print("weigths: {0}".format(weights))

weigths: [0.3829787234042553, 0.3191489361702128, 0.2978723404255319]


In [112]:
def normalize(table):
    result = np.copy(table)
    result[:, 2] /= np.max(result[:, 2])
    return result

In [113]:
def agreement_matrix(table):
  c = np.zeros((len(table), len(table)))

  np.fill_diagonal(c, 1)
  for i in range(len(table)):
      iter_row = table[i]
      for j in range(len(table)):
        if(i != j):
          jiter_row = table[j]

          betters_value = []
          for m in range(len(iter_row)):
            if(iter_row[m] >= jiter_row[m]):
              betters_value.append(m)

          for p in range(len(betters_value)):
            c[i][j] += weights[betters_value[p]] 
  
  print("agreement matrix:\n {}".format(c))
  return c     

In [114]:
def disagreement_matrix(table):
    c = np.zeros((len(table), len(table)))

    np.fill_diagonal(c, -1)
    for i in range(len(table)):
        iter_row = table[i]
        for j in range(len(table)):
            if(i != j):
                jiter_row = table[j]

                betters_value = []
                for m in range(len(iter_row)):
                    if(iter_row[m] <= jiter_row[m]):
                        betters_value.append(m)

                for p in range(len(betters_value)):
                    c[i][j] += weights[betters_value[p]] 

    print("disagreement matrix:\n {}".format(c))
    return c    

In [115]:
def electra_method(pareto_table):
    agr_matrix = agreement_matrix(pareto_table)
    disagr_matrix = disagreement_matrix(pareto_table)
    agr_min, disagr_max = np.amin(agr_matrix, 1), np.amax(disagr_matrix, 1) 
    с_core, d_core = 0.31, 0.7
    print('Agreement core: {}'.format(с_core))
    print('Disagreement core: {}'.format(d_core))
    print('Agreement min: {}'.format(agr_min))
    print('Disagreement max: {0}\n'.format(disagr_max))

    print('Pareto alternatives:')
    for row in pareto_table:
        print(row)

    print('Best alternatives by Electra:')
    for i in range(len(agr_min)):
        if(agr_min[i] > с_core and disagr_max[i] < d_core):
            print('{}'.format(pareto_table[i]))


In [116]:
electra_method(pareto_table)

agreement matrix:
 [[1.         0.61702128 0.68085106]
 [0.68085106 1.         0.68085106]
 [0.70212766 0.31914894 1.        ]]
disagreement matrix:
 [[-1.          0.68085106  0.70212766]
 [ 0.61702128 -1.          0.31914894]
 [ 0.68085106  0.68085106 -1.        ]]
Agreement core: 0.31
Disagreement core: 0.7
Agreement min: [0.61702128 0.68085106 0.31914894]
Disagreement max: [0.70212766 0.61702128 0.68085106]

Pareto alternatives:
[0.4, 0.73, 3.5]
[1.0, 0.4, 3.5]
[0.4, 1.23, 3]
Best alternatives by Electra:
[1.0, 0.4, 3.5]
[0.4, 1.23, 3]
